# Environment

In [ ]:
import sys
from os import path
sys.path.append(path.dirname(path.dirname(path.abspath("Test.ipynb"))))
import time
import logging
from Controllers_Definition import testcontroller1, testcontroller2, testcontroller3, testcontroller4
from datetime import datetime
from FMLC.triggering import triggering
from FMLC.baseclasses import eFMU
from FMLC.stackedclasses import controller_stack

In [ ]:
logger = logging.getLogger(__name__)
'''
import matplotlib.pyplot as plt
%matplotlib inline
'''
logger.setLevel(logging.DEBUG)

# Triggering Class

In [ ]:
ts = {} 
ts['main'] = 0.5 # seconds
ts['print'] = 1 # seconds

trigger_test = triggering(ts)
now_init = time.time()
now = now_init
while now < now_init+2:
    now = time.time()
    if now >= trigger_test.trigger['main']:
        print ('Main triggered', now)
        trigger_test.refresh_trigger('main', now)
    if now >= trigger_test.trigger['print']:
        print ('Print triggered', now)
        trigger_test.refresh_trigger('print', now)

# Controller Base Class (eFMU)

In [ ]:
# Test controller
testcontroller = testcontroller1()
# Get all variables
variables = testcontroller.get_model_variables()
# Makeup some inputs
inputs = {}
for var in variables:
    inputs[var] = 2
# Query controller
print ('Log-message', testcontroller.do_step(inputs=inputs))
print ('Input', testcontroller.input)
print ('Output', testcontroller.output)
print('Output', testcontroller.get_var('output'))

# Controller Stack Class (single-thread/multi-thread)

In [ ]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller1, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'

## Single Thread

In [ ]:
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=False, timeout=2)

# Call query_control 6 times. We should expect 6 records(excluding NaN) for each controller.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
for i in range(6):
    ctrl_stack.query_control(time.time())
for df in ctrl_stack.log_to_df().values():
    display(df)

## Multi Thread

In [ ]:
# Initialize another controller_stack using the mappings above. This time, parallel = True
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller1, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, timeout=2)

# Call query_control 6 times. We should expect there are 6 records(excluding NaN) for each task.
# In multi thread mod, each call of query_control will trigger a computation for one controller within each task. 
# We assign tasks based on input dependency. Since the inputs of mpc1 and control1 depend on the out put of
# forecast1, they are in the same task as forecast1.
for i in range(6):
    ctrl_stack.query_control(time.time())
    time.sleep(1.5)
for df in ctrl_stack.log_to_df().values():
    display(df)